In [ ]:
# Install dependencis in your terminal
# pip install mediapipe opencv-python


In [2]:
# Import all needed packages
import mediapipe as mp
import cv2
import numpy as np
import os
import uuid
import math

In [3]:
# Mediapipe confg variables
mp_drawing = mp.solutions.drawing_utils # Module for rendering the landmarks on video/image
mp_hands = mp.solutions.hands # Module for work with hands - detection and tracking 

In [4]:
# Function for denormalization joint coordinates
def denormalization(hand, hand_joint_num, img_height, img_width):
    normalized_x = hand.landmark[hand_joint_num].x 
    normalized_y = hand.landmark[hand_joint_num].y
    denorm_x = int(min(math.floor(normalized_x * img_width), img_width - 1))
    denorm_y = int(min(math.floor(normalized_y * img_height), img_height - 1))

    return (denorm_x, denorm_y)

In [8]:
# Active WebCam
cap = cv2.VideoCapture(0)
image_height = cap.get(3)
image_width = cap.get(4)

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence= 0.5) as hands:
    while cap.isOpened():
        _, frame = cap.read() # Catches the frame from video

        # Hands detections
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # Change frame's colors into RGB scheme
        # image.flags.writeable = False # Set flag to false
        results = hands.process(image)  # Detections
        # image.flags.writeable = True # Set flagto true
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # Rechange colors
        
        # Draw detections on frame
        if results.multi_hand_landmarks:            
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS)
            cv2.line(image, denormalization(hand, 8, image_width, image_height), denormalization(hand, 4, image_width, image_height), (255, 0, 0), 5)

        
        cv2.imshow('Hand Tracking', cv2.flip(image,1))  # Display frame; Flip display image as mirrored

        # Get last 8 bits from pressed key.
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release() # Release WebCam
    cv2.destroyAllWindows() 



230